In [1]:
import cv2
from os import path
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial import Delaunay
from scipy.linalg import lstsq
from scipy.interpolate import interp2d
import glob

In [2]:
def select_correspondence_points(image, cp_fn):
    """
    Inputs:
        image:  An image to create correspondence points for
        cp_fn:  A file name to save the correspondence points text file as
    Outputs:
        A numpy array of coordinates to chosen correspondence points. Also saves a text file containing
        coordinates of correspondence points to folder containing images.
    """
    
    height, width = image.shape[0], image.shape[1]
    fig = plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(image)
    points = []
    
    
    # When mouse is pressed add coordinates to list
    def onclick(event):
        x, y = event.xdata, event.ydata
        points.append([round(x, 2), round(y, 2)])
        plt.plot(x, y, 'ro')
        plt.annotate(len(points), (x, y), textcoords="offset points", xytext=(0,5))

    
    def onclose(event):
        # Add correspondence points at each corner and midpoint betweeen corners
        # for warping backgrounds
        points.append([0.00, 0.00])
        points.append([width / 2, 0.00])
        points.append([width, 0.00])
        points.append([width, height / 2])
        points.append([width, height])
        points.append([width / 2, height])
        points.append([0.00, height])
        points.append([0.00, height / 2])
        
        # Write the coordinates to a text file    
        with open(cp_fn, "w") as file:
            for point in points:
                file.write("{}\t{}\n".format(point[0], point[1]))

    
    fig.canvas.mpl_connect('button_press_event', onclick)
    fig.canvas.mpl_connect('close_event', onclose)
    
    return points

In [3]:
def text_to_list(file_path):
    """
    Inputs:
        file_path:  The path to a text file containing correspondence points, with each line containing the 
            x and y coordinate of each point separated by a space
    Outputs:
        An nx2 array containing the coordinates of n correspondence points. Each entry is a list of the
        coordinates, [x,y], for a point.
    """
    with open(file_path, 'r') as f:
        cp = []

        for line in f:
            cp.append(line.split())

        for coords in cp:
            coords[0] = float(coords[0])
            coords[1] = float(coords[1])
    
    #convert to numpy array
    cp = np.array(cp)
    
    return cp

In [4]:
def interp_points(p1, p2, t):
    """
    Inputs:
        p1:  A list of x,y coordinates for correspondence points
        p2:  A list of x,y coordinates for correspondence points
        t:   A value in range [0,1] that controls how much of each image contributes to the average shape
    Outputs:
        A list of the interpolated points
    """
    
    # initialize array
    avg_pts = np.zeros(p1.shape)

    # Linearly interpolate between corresponding points in each image
    for n in range(len(avg_pts)):
        avg_pts[n,0] = (1 - t) * p1[n][0] + t * p2[n][0]
        avg_pts[n,1] = (1 - t) * p1[n][1] + t * p2[n][1]
    
    return avg_pts

## Slow Implementation
The slow implementation of compute_affine() and morph() computes a pixel value for each pixel in the average image by using the affine transform to map from the pixel in the average image to the pixels in the morphing images, and computing a weighted weihgted average of those pixel values.

In [5]:
def compute_affine(x1, y1, x2, y2):
    """
    Inputs:
        x1:  x coordinates of correspondence points in  first triangle
        y1:  y coordinates of correspondence points in first triangle
        x2:  x coordiantes of correspondence points in second triangle
        y2:  y coordinates of correspondence points in second triangle
    Outputs:
        Tansformation matrix that produces an affine warp from first triangle to second triangle
    """
    
    A = np.zeros([6,6])
    b = np.zeros([6,1])
    
    c = 0
    for i in range(3):
        A[c,:] = [x1[i], y1[i], 1, 0, 0, 0]
        b[c] = x2[i]
        c = c + 1
        A[c,:] = [0, 0, 0, x1[i], y1[i], 1]
        b[c] = y2[i]
        c = c + 1
    
    params = lstsq(A,b)
    affine = np.zeros([3,3])
    affine[0,:] = [params[0][0], params[0][1], params[0][2]]
    affine[1,:] = [params[0][3], params[0][4], params[0][5]]
    affine[2,2] = 1
    
    return affine

In [18]:
def morph(im1, im2, cp1, cp2, t):
    """
    Inputs:
        im1:  The first image
        im2:  The second image
        cp1:  Correspondence points for the first image
        cp2:  Correspondence points for the second image
        t:  A fraction between 0 and 1 that defines the intermediate shape and the amount to cross dissolve
    Outputs:
        An image that is a warp between image 1 and image 2, with the amount of warp defined
        by the fraction t.
    """
    # Compute the average of corresponding points between images
    avg_pts = interp_points(cp1, cp2, t)
    
    # Compute the Delauney triangulation of the average points
    tri = Delaunay(avg_pts)

    out_im = np.zeros(im1.shape)
    
    for n in range(tri.simplices.shape[0]):
        # obtain coordinates of traingles in image 1
        x1_im1 = cp1[tri.simplices[n,0],0]
        y1_im1 = cp1[tri.simplices[n,0],1]
        x2_im1 = cp1[tri.simplices[n,1],0]
        y2_im1 = cp1[tri.simplices[n,1],1]
        x3_im1 = cp1[tri.simplices[n,2],0]
        y3_im1 = cp1[tri.simplices[n,2],1]

        # obtain coordinates of traingles in image 2
        x1_im2 = cp2[tri.simplices[n,0],0]
        y1_im2 = cp2[tri.simplices[n,0],1]
        x2_im2 = cp2[tri.simplices[n,1],0]
        y2_im2 = cp2[tri.simplices[n,1],1]
        x3_im2 = cp2[tri.simplices[n,2],0]
        y3_im2 = cp2[tri.simplices[n,2],1]

        # obtain coordinates of traingles in average image
        x1_avg = avg_pts[tri.simplices[n,0],0]
        y1_avg = avg_pts[tri.simplices[n,0],1]
        x2_avg = avg_pts[tri.simplices[n,1],0]
        y2_avg = avg_pts[tri.simplices[n,1],1]
        x3_avg = avg_pts[tri.simplices[n,2],0]
        y3_avg = avg_pts[tri.simplices[n,2],1]

        pts_im1 = np.float32([[x1_im1, y1_im1],
                              [x2_im1, y2_im1],
                              [x3_im1, y3_im1]])

        pts_im2 = np.float32([[x1_im2, y1_im2],
                              [x2_im2, y2_im2],
                              [x3_im2, y3_im2]])

        pts_avg = np.float32([[x1_avg, y1_avg],
                              [x2_avg, y2_avg],
                              [x3_avg, y3_avg]])

        # Compute the affine transformation matrix between corresponding triangles in image 1 and average image
        A_im1 = compute_affine(pts_im1[:,0], pts_im1[:,1], pts_avg[:,0], pts_avg[:,1])

        # Compute the affine transformation matrix between corresponding triangles in image 1 and average image
        A_im2 = compute_affine(pts_im2[:,0], pts_im2[:,1], pts_avg[:,0], pts_avg[:,1])

        # inverse to convert points from average image to image 1 and image 2
        A_im1_inv = np.linalg.inv(A_im1)
        A_im2_inv = np.linalg.inv(A_im2)
        
        # Find the extents of the current average triangle
        xmin = np.floor(min(pts_avg[i,0] for i in range(3)))
        ymin = np.floor(min(pts_avg[i,1] for i in range(3)))
        xmax = np.ceil(max(pts_avg[i,0] for i in range(3)))
        ymax = np.ceil(max(pts_avg[i,1] for i in range(3)))

        # For all pixels in a bounding box created by the max and min coordinate of simpleces
        # of the average triangle
        for row in range(int(ymin), int(ymax)):
            for col in range(int(xmin), int(xmax)):
                
                # Check if current pixel is in the triangle
                if tri.find_simplex([col, row]) == n:
                    
                    # Use inverse affine transform to find value of pixel in image 1 and image 2
                    point_im1 = np.dot(A_im1_inv, np.transpose([col, row, 1]))
                    point_im2 = np.dot(A_im2_inv, np.transpose([col, row, 1]))
                    val_im1 = im1[int(point_im1[1]), int(point_im1[0]), :]
                    val_im2 = im2[int(point_im2[1]), int(point_im2[0]), :]
                    
                    # take the weighted average of the the two pixesl and apply them to the average image
                    val = (1-t) * val_im1 + t * val_im2
                    out_im[row, col, :] = val
        
    return out_im

## Fast Implementation
The fast implementation of compute_affine() and morph() uses cv2.warpAffine to warp each image to the average shape, and then the average triangle is isolated with a mask and place in the output image.

In [19]:
def create_morph_video(video_fn='morph.m', scale=1, number_frames=60, fps=30):
    """
    Inputs:
        cps:  A list containing correspondence points for each image pair in the same order as the
            images should appear in the video
        video_fn:  An optional name for the output video file
        scale:  An optional scale to improve processing speed. Use a number less than zero
            to speed up processing
        number_frames:  The number of desired frames per image morph
        fps:  The desired frames per second in the output video
    Outputs:
        A video of a morphing between all of the images in the image_file_location with morphing defined by
        the control points
    """
    # create a list of all images
    image_fns = sorted(glob.glob('input/*.jpg') + glob.glob('input/*.jpeg'))
    images = [cv2.cvtColor(cv2.imread(image_fns[n]), cv2.COLOR_BGR2RGB) for n in range(len(image_fns))]
    
    # find the average size of all images in list
    avg_height = round(sum(image.shape[0] for image in images) / len(image_fns))
    avg_width = round(sum(image.shape[1] for image in images) / len(image_fns))
    
    # resize all images to average size, and scale down for faster processing
    im_resize = [cv2.resize(image, (round(avg_width * scale), round(avg_height * scale))) for image in images]
    
    # create a list of correspondence points from text file
    cps = [text_to_list(fn) for fn in sorted(glob.glob('input/*.txt'))]
    
    # If text files do not exist select correspondence points and rerun
    if not cps:
        print("Select correspondence points and rerun")
        for i in range(len(im_resize)):
            cp = select_correspondence_points(im_resize[i], 'input/image{}.txt'.format(i+1))
    
    # loop through each image in the input folder, for each pair of image, loop n times where n is the
    # desired number of frames. Increment the weighting variable passed to morph() function in each loop
    for i in range(len(cps) - 1):
        for t in range(number_frames):
            out_im = morph(im_resize[i], im_resize[i+1], cps[i], cps[i+1], (t+1)/number_frames)
            fname = 'output/warp{}_frame_{}.jpg'.format(i+1, t+1)
            cv2.imwrite(fname, cv2.cvtColor(np.array(out_im, dtype='float32'), cv2.COLOR_RGB2BGR))
            
    # Create video write object
    video = cv2.VideoWriter('{}.mp4'.format(video_fn), cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 
                            fps=fps, frameSize=(im_resize[0].shape[1], im_resize[0].shape[0]))

    # write frames to video
    for frame in glob.glob('output/*.jpg'):
        video.write(cv2.imread(frame))

    # release video write object
    video.release()

In [ ]:
create_morph_video(video_fn='presidents.mp4', scale=1/3)